# Assignment 5, Question 6: Data Transformation

**Points: 20**

Transform and engineer features from the clinical trial dataset.

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import utilities
from q3_data_utils import load_data, clean_data, transform_types, create_bins, fill_missing

df = load_data('data/clinical_trial_raw.csv')
print(f"Loaded {len(df)} patients")

# Prewritten visualization functions for transformation analysis
def plot_distribution(series, title, figsize=(10, 6)):
    """
    Create a histogram of a numeric series.
    
    Args:
        series: pandas Series with numeric data
        title: Chart title
        figsize: Figure size tuple
    """
    plt.figure(figsize=figsize)
    series.hist(bins=30)
    plt.title(title)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.show()

def plot_value_counts(series, title, figsize=(10, 6)):
    """
    Create a bar chart of value counts.
    
    Args:
        series: pandas Series with value counts
        title: Chart title
        figsize: Figure size tuple
    """
    plt.figure(figsize=figsize)
    series.plot(kind='bar')
    plt.title(title)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

Loaded 10000 patients


## Part 1: Type Conversions (5 points)

1. Convert 'enrollment_date' to datetime using the `transform_types()` utility
2. Convert categorical columns ('site', 'intervention_group', 'sex') to category dtype
3. Ensure all numeric columns are proper numeric types
4. Display the updated dtypes

In [3]:
# TODO: Type conversions
# 1. Use transform_types() to convert enrollment_date to datetime

df = transform_types(df, {'enrollment_date': 'datetime'})

# 2. Convert categorical columns ('site', 'intervention_group', 'sex') to category dtype

categorical_columns = ['site', 'intervention_group', 'sex']
for col in categorical_columns:
    df = transform_types(df, {col: 'category'})

# 3. Ensure all numeric columns are proper numeric types

numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
for col in numeric_columns:
    df = transform_types(df, {col: 'numeric'})

# 4. Display the updated dtypes using df.dtypes

print("Updated data types:")
print(df.dtypes)

Updated data types:
patient_id                    object
age                            int64
sex                         category
bmi                          float64
enrollment_date       datetime64[ns]
systolic_bp                  float64
diastolic_bp                 float64
cholesterol_total            float64
cholesterol_hdl              float64
cholesterol_ldl              float64
glucose_fasting              float64
site                        category
intervention_group          category
follow_up_months               int64
adverse_events                 int64
outcome_cvd                   object
adherence_pct                float64
dropout                       object
dtype: object


## Part 2: Feature Engineering (8 points)

Create these new calculated columns:

1. `cholesterol_ratio` = cholesterol_ldl / cholesterol_hdl
2. `bp_category` = categorize systolic BP:
   - 'Normal': < 120
   - 'Elevated': 120-129
   - 'High': >= 130
3. `age_group` using `create_bins()` utility:
   - Bins: [0, 40, 55, 70, 100]
   - Labels: ['<40', '40-54', '55-69', '70+']
4. `bmi_category` using standard BMI categories:
   - Underweight: <18.5
   - Normal: 18.5-24.9
   - Overweight: 25-29.9
   - Obese: >=30

In [4]:
# TODO: Calculate cholesterol ratio
df['cholesterol_ratio'] = df['cholesterol_ldl'] / df['cholesterol_hdl']

In [5]:
# TODO: Categorize blood pressure
def categorize_bp(systolic):
    if systolic < 120:
        return 'Normal'
    elif systolic < 130:
        return 'Elevated'
    else:
        return 'High'
df['bp_category'] = df['systolic_bp'].apply(categorize_bp)

**Note:** The `create_bins()` function has an optional `new_column` parameter. If you don't specify it, the new column will be named `{original_column}_binned`. You can use `new_column='age_group'` to give it a custom name.


In [6]:
# TODO: Create age groups
df = create_bins(
    df,
    column='age',
    bins=[0, 40, 55, 70, 100],
    labels=['<40', '40-54', '55-69', '70+'],
    new_column='age_group'
)

In [7]:
# TODO: Create BMI categories
def categorize_bmi(bmi):
    if bmi < 18.5:
        return 'Underweight'
    elif bmi < 25:
        return 'Normal'
    elif bmi < 30:
        return 'Overweight'
    else:
        return 'Obese'

df['bmi_category'] = df['bmi'].apply(categorize_bmi)

print("New columns created:")
print(f"cholesterol_ratio")
print(f"bp_category")
print(f"age_group")
print(f"bmi_category")

New columns created:
cholesterol_ratio
bp_category
age_group
bmi_category


## Part 3: String Cleaning (2 points)

If there are any string columns that need cleaning:
1. Convert to lowercase
2. Strip whitespace
3. Replace any placeholder values

In [8]:
# TODO: String cleaning
string_columns = df.select_dtypes(include=['object']).columns.tolist()
for col in string_columns:
    df[col] = df[col].str.lower().str.strip()
    df[col] = df[col].replace(['n/a', 'na', 'none', 'null', ''], np.nan)

## Part 4: One-Hot Encoding (5 points)

Create dummy variables for categorical columns:
1. One-hot encode 'intervention_group' using `pd.get_dummies()`
2. One-hot encode 'site'
3. Drop the original categorical columns
4. Show the new shape and column names

In [9]:
# TODO: One-hot encoding
# 1. One-hot encode 'intervention_group' using pd.get_dummies()
intervention_dummies = pd.get_dummies(df['intervention_group'], prefix='intervention')

# 2. One-hot encode 'site'
site_dummies = pd.get_dummies(df['site'], prefix='site')

# 3. Drop the original categorical columns
df = df.drop(['intervention_group', 'site'], axis=1)
df = pd.concat([df, intervention_dummies, site_dummies], axis=1)

# 4. Show the new shape and column names
print(f"New shape: {df.shape}")
print(f"New columns: {list(df.columns)}")

New shape: (10000, 80)
New columns: ['patient_id', 'age', 'sex', 'bmi', 'enrollment_date', 'systolic_bp', 'diastolic_bp', 'cholesterol_total', 'cholesterol_hdl', 'cholesterol_ldl', 'glucose_fasting', 'follow_up_months', 'adverse_events', 'outcome_cvd', 'adherence_pct', 'dropout', 'cholesterol_ratio', 'bp_category', 'age_group', 'bmi_category', 'intervention_  CONTROL  ', 'intervention_  Contrl  ', 'intervention_  Control  ', 'intervention_  TREATMENT A  ', 'intervention_  TREATMENT B  ', 'intervention_  Treatmen A  ', 'intervention_  Treatment  B  ', 'intervention_  Treatment A  ', 'intervention_  Treatment B  ', 'intervention_  TreatmentA  ', 'intervention_  control  ', 'intervention_  treatment a  ', 'intervention_  treatment b  ', 'intervention_CONTROL', 'intervention_Contrl', 'intervention_Control', 'intervention_TREATMENT A', 'intervention_TREATMENT B', 'intervention_Treatmen A', 'intervention_Treatment  B', 'intervention_Treatment A', 'intervention_Treatment B', 'intervention_Tre

## Part 5: Save Transformed Data

Save the fully transformed dataset to `output/q6_transformed_data.csv`

In [11]:
# TODO: Save transformed data
# df_transformed.to_csv('output/q6_transformed_data.csv', index=False)
df.to_csv('output/q6_transformed_data.csv', index=False)
print(f"Transformed data saved to 'output/q6_transformed_data.csv'")

Transformed data saved to 'output/q6_transformed_data.csv'
